In [ ]:
# nltk.download('stopwords')
# nltk.download('wordnet')

In [ ]:
# !python -m spacy download en_core_web_sm
# !python -m spacy download en
# !python -m spacy download en_core_web_md

In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import requests
import re
import nltk
import textcleaner
import pickle
import spacy
import jieba

from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize, sent_tokenize
from textblob import TextBlob
import textcleaner as tc
from nltk.stem import WordNetLemmatizer 
import re
import spacy

## Data prep

### Initial data cleaning

In [4]:
# Load scraped job description data

file_list = [
'data_110619_seattle_0to1k.pkl',
'data_110619_seattle_1kto2k.pkl',
'data_110619_seattle_2kto3k.pkl',
'data_110619_seattle_neg_5.pkl',
'data_110619_seattle_3kto4k.pkl',
'data_110619_seattle_4ktoend.pkl',
'data_110619_bellevue_0to1k.pkl',
'data_110619_bellevue_1to2k.pkl',
'data_110619_bellevue_2to3k.pkl',
'data_110619_bellevue_3toend.pkl'            ]
jobs_target = pd.DataFrame()

for file in file_list:
    
    folder = '/Users/greenapple/project4/data/interim/'
    path = os.path.join(folder, file)
    
    pickling_out = open(path, 'rb')
    df_piece = pickle.load(pickling_out)
    jobs_target = pd.concat([jobs_target, df_piece]) 
    
    if file == 'data_110619_seattle_neg.pkl':
        jobs_target['type'] = 'negative'
    else:
        jobs_target['type'] = 'positive'
                        

/Users/greenapple/anaconda3/envs/project4e/lib/python3.7/site-packages/ipykernel_launcher.py:23: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [5]:
jobs_target.shape

(8699, 7)

In [6]:
# Remove duplicates and data rows that were inserted by indeed
jobs_target.drop_duplicates(inplace=True)
a = jobs_target.shape
jobs_target.drop(jobs_target.loc[jobs_target.company_name == 'Seen by Indeed'].index, inplace=True)
jobs_target.reset_index(drop=True, inplace=True)
b = jobs_target.shape
print(a, b)

(8675, 7) (5058, 7)


In [17]:
# Collect data for the rows replaced by indeed
replaced_data_0to1k = data_110619_seattle_set1.loc[data_110619_seattle_set1.company_name=='Seen by Indeed']

In [21]:
# Load metis resumes
pickling_out = open('/Users/greenapple/project4/data/resumes/resumes_110719', 'rb')
resumes_l = pickle.load(pickling_out)

resumes = pd.DataFrame(resumes_l, columns = ['description'])
resumes.shape

(5, 1)

### Functions

In [72]:
# Pre-process text
def pre_process(text):
    '''
    Takes a string, returns a cleaned string.
    '''
    text_sub = re.sub('[/.\n:-]', ' ', text)  # Remove seaparators of dif kind
    text_sub = re.sub('[,\(\)]', '', text_sub).lower()  # Remove "," and capital letters
    text_sub = re.sub('[^a-z\s]', '', text_sub) # Keep alphanumeric characters only
#     re.sub('[%s]' % re.escape(string.punctuation), ' ', my_text)

    return text_sub

In [22]:
from gensim.parsing.preprocessing import preprocess_string

In [69]:
# Pre-process text
from gensim.parsing.preprocessing import preprocess_string, strip_numeric, strip_short, stem_text, strip_tags, strip_punctuation, strip_multiple_whitespaces, remove_stopwords

def gensim_preprocess(text):
    '''
    Takes a string, returns a list of cleaned strings.
    '''
    filters = [
        strip_tags,
        strip_punctuation, 
        strip_multiple_whitespaces, 
        strip_numeric, 
        remove_stopwords, 
        strip_short,
        lambda x: x.lower()]
    
    text_proc = preprocess_string(text, filters)
    return text_proc

In [22]:
# Tokenize text
def tokenize_lemma(text):
        
    # processer = spacy.load('en')
    tokenizer = spacy.load('en_core_web_sm')

    text_obj = tokenizer(text)
    text_str = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
    return text_str

In [203]:
# Tokenize text: nouns

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_noun(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_noun = ' '.join([token.text for token in text_obj if token.pos_=='NOUN'])
  
    return text_noun

In [205]:
# Tokenize text: lemma and nouns

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_noun_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_noun_lemma = ' '.join([token.lemma_ for token in text_obj if token.pos_=='NOUN'])
   
  
    return text_noun_lemma

In [212]:
# Tokenize text: lemma

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_lemma(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_lemma = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
  
    return text_lemma

In [213]:
# Tokenize text: words

tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_word(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_text = ' '.join([token.text for token in text_obj if not token.is_stop])
  
    return text_text

In [ ]:
# DOES NOT WORK
# Tokenize text: more choices

# processer = spacy.load('en')
tokenizer = spacy.load('en_core_web_sm')
    
def tokenize_multi(text):
        
    text_obj = tokenizer(text, disable=['parser', 'ner'])
    
    text_noun = ' '.join([token.text for token in text_obj if token.pos_=='NOUN'])
    text_noun_lemma = ' '.join([token.lemma_ for token in text_obj if token.pos_=='NOUN'])
    text_lemma = ' '.join([token.lemma_ for token in text_obj if not token.is_stop])
    text_text = ' '.join([token.text for token in text_obj if not token.is_stop])
  
    return text_noun, text_noun_lemma, text_lemma, text_text

In [225]:
# Vectorize: tf-idf
def tf_idf(text1, text2):
    vectorizer = TfidfVectorizer(ngram_range=(1, 2),               # vectorizer
                        stop_words='english', 
                        token_pattern="\\b[a-z][a-z]+\\b")
    X_tfidf = vectorizer.fit_transform(text1).toarray()            # vectorize text1
    Z_tfidf = vectorizer.transform(text2).toarray()                # vectorize text1
    
    
#     print('text1{}'.format(pd.DataFrame(X_tfidf, columns=tfidf.get_feature_names()).head(5)))
#     print('text2{}'.format(pd.DataFrame(Z_tfidf, columns=tfidf.get_feature_names()).head(5)))
    
    return X_tfidf, Z_tfidf

In [226]:
# Vectorize: count
def count_(text1, text2):
    vectorizer = CountVectorizer(stop_words='english')
    X_vect = vectorizer.fit_transform(text1).toarray()            # vectorize text1
    Z_vect = vectorizer.transform(text2).toarray()                # vectorize text1
    
    
#     print('text1{}'.format(pd.DataFrame(X_tfidf, columns=tfidf.get_feature_names()).head(5)))
#     print('text2{}'.format(pd.DataFrame(Z_tfidf, columns=tfidf.get_feature_names()).head(5)))
    
    return X_vect, Z_vect

In [231]:
# Reduce dimensionality:
def lsa(X, Z, components):
    lsa = TruncatedSVD(components)
    X_lsa = lsa.fit_transform(X)
    Z_lsa = lsa.transform(Z)
  
    return X_lsa, Z_lsa

In [88]:
def similarity(X_red, Z_red):
    score = cosine_similarity(X_red, Z_red).round(2)
    
    column_labels = ['Resume'+str(i) for i in range(1, R_lsa.shape[0]+1)]
    row_labels = ['Job'+str(i) for i in range(1, Z_lsa.shape[0]+1)]
   
    similarity_df = pd.DataFrame(score, columns=column_labels, index=row_labels)
#     print(similarity_df)
        
    return similarity_df

### Clean text, tokenize

In [122]:
# Clean and tokenize
jobs['clean_text'] = jobs['description'].apply(pre_process) # clean



KeyboardInterrupt: 

In [125]:
# Clean
jobs['clean_text'] = jobs['description'].apply(pre_process) # clean

# Save cleaned text
pickling_in = open('/Users/greenapple/project4/data/processed/jobs_clean.pkl', 'wb')
pickle.dump(jobs, pickling_in)

In [215]:
# Tokenize:
jobs['noun'] = jobs['clean_text'].apply(tokenize_noun)
jobs['noun_lemma'] = jobs['clean_text'].apply(tokenize_noun_lemma)
jobs['lemma'] = jobs['clean_text'].apply(tokenize_lemma)
jobs['word'] = jobs['clean_text'].apply(tokenize_word)

In [216]:
jobs.columns

Index(['company_name', 'description', 'job_title', 'link', 'location',
       'salary', 'type', 'clean_text', 'noun', 'noun_lemma', 'lemma', 'word'],
      dtype='object')

In [217]:
# Save tokenized text
pickling_in = open('/Users/greenapple/project4/data/processed/jobs_tokenized.pkl', 'wb')
pickle.dump(jobs, pickling_in)

### Vectorize

In [229]:
# Vectorize: Count and TF-IDF
job_tf_idf_noun, _ = tf_idf(jobs['noun'], jobs['noun'])  # TF-IDF Vectorizer
job_count_noun, _ = count_(jobs['noun'], jobs['noun'])  # Count Vectorizer

In [ ]:
# Word2vec

In [ ]:
# GloVe

### Reduce dimentionality

In [405]:
# LSA: noun
vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                            max_df = 0.9,
                            min_df = 0.05,
                            stop_words = ['yearexperience', 'year', 'years', 'porch', 'work', 'home', 'term'])

noun_vc = vectorizer.fit_transform(jobs.noun)

len(vectorizer.vocabulary_)

1223

In [410]:
components = 20
lsa = TruncatedSVD(components)


noun_topics = lsa.fit_transform(noun_vc.toarray())  # reduce dimensionality

lsa.explained_variance_ratio_

array([0.02064644, 0.09040681, 0.06895247, 0.05957077, 0.05476894,
       0.05341638, 0.0498317 , 0.04515979, 0.04129339, 0.04040844,
       0.0344565 , 0.03288877, 0.03081242, 0.02632454, 0.02560626,
       0.02471311, 0.02374627, 0.01976032, 0.01691725, 0.01350298])

In [416]:
# pd.DataFrame(noun_vc.toarray(), columns=vectorizer.get_feature_names()).head(10)

In [412]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [413]:
display_topics(lsa, vectorizer.get_feature_names(), 20)


Topic  0
data, team, business, science, data science, engineering, design, solutions, models, problems, analysis, ability, machine, customer, projects, services, management, time, project, skills

Topic  1
engineering, design, water, project, site, client, construction, drawings, engineer, transportation, traffic, firm, clients, specifications, development, management, position, employees, study, staff

Topic  2
warehouse, database, data, data technologies, azure, pipelines data, databricks, technologies, pipelines, platform, manufacturing, solutions, solutions fortune, process warehouse, excell com, azure visas, relationships consultants, team database, database background, data azure

Topic  3
cell, patients, medicine, therapy, cell therapy, study, system, cancer, trial, receptors, research, patient, patient experience, therapy patient, information, ability, partners, trials, studies, problem

Topic  4
cell, patients, client, medicine, cell therapy, therapy, cancer, manufacturing, r

In [429]:
# NMF

nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(noun_vc)

display_topics(nmf_model, vectorizer.get_feature_names(), 5)


Topic  0
datum, data, data scientist, business, analytic

Topic  1
engineering, design, water, drawing, construction

Topic  2
datum, warehouse, process warehouse, database, datum technology

Topic  3
patient, cell, therapy, medicine, cancer

Topic  4
datum, datum science, hand, datum scientist, team

Topic  5
client, project, candidate, sector, position

Topic  6
query, ambiguity, guideline, search, change

Topic  7
study, trial, document, vendor, execution

Topic  8
fish, player, visualization, datum, business

Topic  9
client, program, program manager, manager, change


In [435]:
# NMF: noun_lemma
vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                            max_df = 0.8,
                            min_df = 0.05,
                            stop_words = ['yearexperience', 'year', 'years', 'porch', 'work', 'home', 
                                          'term', 'datum', 'hand', 'science'])

noun_vc = vectorizer.fit_transform(jobs.noun_lemma)

len(vectorizer.vocabulary_)


1142

In [436]:
nmf_model = NMF(20)
doc_topic = nmf_model.fit_transform(noun_vc)

display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
engineering, water, design, utility, firm, construction, site, drawing, grading, survey

Topic  1
scientist, customer, learning, model, business opportunity, revenue, statistic, machine learning, product, homeowner

Topic  2
warehouse, process warehouse, database, databrick, azure, pipeline, solution, technology, process, format auditing

Topic  3
fish, player, visualization, analyst, business, creativity, stakeholder, talent, business stakeholder, type time

Topic  4
patient, cell, therapy, medicine, cancer, cell therapy, receptor, manufacturing, patient experience, therapy patient

Topic  5
client, program, program manager, manager, change, transportation, customer, success, service, client relationship

Topic  6
query, ambiguity, guideline, change, search, user, entity, research, research web, guideline ambiguity

Topic  7
study, trial, vendor, execution, study trial, protocol, study site, management, travel, scope

Topic  8
search, analytic, data scientist, data, technolo

In [437]:
jobs.columns

Index(['company_name', 'description', 'job_title', 'link', 'location',
       'salary', 'type', 'clean_text', 'noun', 'noun_lemma', 'lemma', 'word'],
      dtype='object')

In [454]:
vectorizer = TfidfVectorizer(ngram_range=(1, 2),
                            max_df = 0.6,
                            min_df = 0.05,
                            stop_words = ['yearexperience', 'year', 'years', 'porch', 'work', 'home', 
                                          'term', 'datum', 'hand', 'science', 'other', 're', 'sex'])

noun_lemma_vc = vectorizer.fit_transform(jobs.noun_lemma)

len(vectorizer.vocabulary_) 


1124

In [455]:
components = 20
lsa_n_l = TruncatedSVD(components)


noun_topics = lsa_n_l.fit_transform(noun_vc.toarray())  # reduce dimensionality

lsa_n_l.explained_variance_ratio_

array([0.00869464, 0.08902728, 0.071433  , 0.06354637, 0.05619893,
       0.05441557, 0.05237732, 0.04665867, 0.04360922, 0.03971836,
       0.03741997, 0.03478475, 0.03103652, 0.0270211 , 0.02460624,
       0.02390971, 0.02232453, 0.01892156, 0.01646111, 0.01380205])

In [456]:
display_topics(lsa_n_l, vectorizer.get_feature_names(), 10)


Topic  0
provider center, business analyst, client job, engineering design, design site, customer problem, simulation, modeling machine, optimization, engineer passion

Topic  1
sector building, modeling machine, story, leukemia, customer problem, business owner, salary, management software, management system, business analyst

Topic  2
writing, database scientist, product treatment, azure, day, thing industry, plan drawing, product owner, specialist associate, database

Topic  3
patient vision, cell manufacturing, support principle, flexibility business, user acquisition, timeline, world medicine, cancer accomplishment, medicine system, position communication

Topic  4
patient vision, cell manufacturing, timeline, medicine system, cancer accomplishment, center, user acquisition, regression, client job, project management

Topic  5
client job, project management, change assignment, service solution, receptor, ambiguity, environment service, project team, client service, visit excell



IndexError: list index out of range

In [ ]:
# LDA

In [90]:
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

,Resume1,Resume2,Resume3,Resume4,Resume5
Job1,0.54,0.61,0.47,0.48,0.52
Job2,0.99,1.00,0.98,0.95,0.99
Job3,0.81,0.79,0.89,0.68,0.83
Job4,0.54,0.61,0.47,0.48,0.52
Job5,1.00,0.99,0.99,0.97,1.00
